# Dog Classifier - Simple

This week, we will build a simple neural network to classify different dog breeds!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

from net import DogNet
from PIL import Image
from stanford_dogs_data import DogsDataset
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

## Data Preparation

In [ ]:
def load_data(plot=True, batch_size=32):
  transform = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # means, stds
                                  ])
  train_dataset = DogsDataset(root="./data", train=True, cropped=False, transform=transform, download=True)
  test_dataset = DogsDataset(root="./data", train=False, cropped=False, transform=transform, download=True)

  train_loader = # TODO
  test_loader = # TODO

  classes = train_dataset.classes

  if plot:
    samples = random.sample(range(len(train_dataset)), batch_size)
    images = [train_dataset[i][0].detach().numpy().transpose(1, 2, 0) for i in samples]
    labels = [train_dataset[i][1] for i in samples]

    fig = plt.figure(figsize=(batch_size/4 + 5, batch_size/4 + 5))  
    for idx in np.arange(batch_size):
        ax = fig.add_subplot(batch_size/8, 8, idx+1, xticks=[], yticks=[])
        ax.imshow((images[idx] / 2) + 0.5)
        ax.set_title(classes[labels[idx]], {'fontsize': batch_size/5}, pad=0.4)
    plt.tight_layout(pad=1, w_pad=0, h_pad=0)
    plt.show()

  return train_loader, test_loader, classes

In [ ]:
# Step 1 - Load the Stanford Dogs Dataset
train_loader, test_loader, classes = load_data()

## Neural Network Training

In [ ]:
"""
Build and train a model using given dataloader and provided hyperparameters.
"""
def train(dl, lr=0.01, momentum=0.9, num_epochs=5, save=True):
  # set device
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # create model
  model = DogNet()
  model = model.to(device)

  # initialize optimizers
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

  # train model
  model.train()
  for epoch in range(num_epochs):
    running_loss = 0.0 # keep track of total loss in epoch
    for i, (ims, labels) in enumerate(dl):
      # move data to appropriate device
      ims, labels = ims.to(device), labels.to(device)

      # zero parameter gradients
      # TODO

      # forward pass
      # TODO

      # compute loss
      # TODO

      # backward pass
      # TODO

      # update parameters
      # TODO

      # print statistics
      running_loss += loss.item()
      if i % 45 == 44:
        print(f"Epoch: {epoch}, Batch Idx: {i}, Loss: {round(running_loss / 45, 3)}")
        running_loss = 0.0

  if save:
    torch.save(model.state_dict(), "./dogNet.pth")

  return model

In [ ]:
# Step 2 - Train your neural network on the training data
model = train(train_loader)

## Neural Network Testing

In [ ]:
"""
Test provided model on provided data in dataloader.
"""
def test(model, dl):
  # set device
  device = "cuda" if torch.cuda.is_available() else "cpu"

  # initialize variables
  correct, total = 0, 0
  class_correct, class_total = list(0. for i in range(120)), list(0. for i in range(120))

  # set to evaluation mode
  model.eval()

  # ensure that a backward pass won't modify the gradient
  with torch.no_grad():
    for ims, labels in dl:
      # move data to appropriate device
      ims, labels = ims.to(device), labels.to(device)

      # forward pass to compute prediction of batch
      # TODO

      # get predicted class from outputs
      # TODO

      # compute overall correctness (across classes)
      correct += # TODO
      total += labels.size(0)

      # compute correctness per class
      c = (predicted == labels).squeeze()
      for i in range(c.shape[0]):
        class_correct[labels[i]] += c[i].item()
        class_total[labels[i]] += 1

  print(f"Overall Accuracy: {100 * (correct / total)}")
  for i in range(120):
    print(f"\tAccuracy of {classes[i]}: {round(100 * (class_correct[i] / class_total[i]), 4)}")

In [ ]:
# Step 3 - Evaluate your model on test data
test(model, test_loader)

## Neural Network Prediction

In [ ]:
 """
Open image from given filepath, and use provided model to predict
final classification decision.
"""
def predict(fname, model):
  # load image and display
  # TODO
  display(im)

  # resize and normalize
  transform = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # means, stds
                                  ])
  # apply transform to image
  # TODO

  # detect device and move to appropriate device
  device = "cuda" if torch.cuda.is_available() else "cpu"
  im = im.to(device)
  model.to(device)

  # forward pass to get prediction
  # TODO

  # get predicted class from outputs
  # TODO

  # print final prediction
  print(f"Predicted {classes[predicted]}")

## Load Existing Model

In [ ]:
""" 
Given filename, loads existing neural network.
"""
def load_model(fname):
  model = DogNet()
  model.load_state_dict(torch.load(fname))
  return model

In [ ]:
# Step 4 - use model for prediction
model = load_model("dogNet.pth") 
predict("frankie.png", model)